# 2D Frozen-in Magnetic Flux (Rm >> 1)

**Validation case**: 2D magnetic flux advection in the ideal MHD limit

## Learning Objectives

After completing this notebook, you will understand:

1. Alfven's frozen-in flux theorem and its physical meaning
2. The magnetic Reynolds number and what Rm >> 1 means physically
3. How magnetic flux is conserved during 2D plasma expansion
4. The analytic solution for flux conservation with velocity shear
5. How to validate numerical MHD solvers against flux conservation in 2D

---
## 1. Physics Background

### The Magnetic Induction Equation (Revisited)

The evolution of magnetic field in a conducting plasma is governed by:

$$\frac{\partial \mathbf{B}}{\partial t} = \nabla \times (\mathbf{v} \times \mathbf{B}) + \eta \nabla^2 \mathbf{B}$$

In the **ideal MHD limit** ($\eta \to 0$, or equivalently $R_m \gg 1$), this becomes:

$$\frac{\partial \mathbf{B}}{\partial t} = \nabla \times (\mathbf{v} \times \mathbf{B})$$

This seemingly simple equation has a profound consequence: **Alfvén's theorem**.

### Alfvén's Frozen-in Flux Theorem

**Theorem**: In ideal MHD, the magnetic flux through any surface moving with the plasma is conserved.

Mathematically, if $S(t)$ is a surface that moves with the fluid:

$$\frac{d}{dt} \int_{S(t)} \mathbf{B} \cdot d\mathbf{A} = 0$$

This means:
- Magnetic field lines are "frozen" into the plasma
- Field lines move with the fluid velocity
- Field lines cannot break or reconnect (in ideal MHD)

### Physical Picture

Imagine magnetic field lines as elastic threads sewn into a fluid:
- When the fluid stretches, the threads stretch with it
- When the fluid compresses, the threads bunch together
- The threads cannot slip through the fluid or break

### Our Test Case: 2D Radial Expansion with Axial Structure

We consider an annular plasma with:
- Toroidal field $B_\phi$ with Gaussian axial profile: $B_\phi(r, z) = B_0 \exp(-z^2/2\sigma_z^2)$
- Uniform radial expansion velocity $v_r = v_0$

**Geometry**: Cylindrical coordinates $(r, \phi, z)$ with the field in the $\phi$ direction.

**Initial condition**: $B_\phi$ with Gaussian z-dependence in the annular region.

**2D Flux conservation**: The toroidal flux through any (r, z) slice must move with the plasma:

$$\frac{D\Phi}{Dt} = 0 \quad \text{(Lagrangian derivative)}$$

### 2D Analytic Solution

For a fluid element initially at radius $r_0$, its position at time $t$ is:

$$r(t) = r_0 + v_r t$$

The z-coordinate remains unchanged (no axial flow): $z(t) = z_0$.

Flux conservation for each (r, z) point gives:

$$B_\phi(r, z, t) \cdot r = B_\phi(r_0, z_0, 0) \cdot r_0$$

With the initial 2D profile $B_\phi(r_0, z, 0) = B_0 \exp(-z^2/2\sigma_z^2)$:

$$B_\phi(r, z, t) = B_0 \frac{r_0}{r_0 + v_r t} \exp\left(-\frac{z^2}{2\sigma_z^2}\right)$$

Key features:
- **Radial decay**: $B_\phi \propto r_0/(r_0 + v_r t)$ from flux conservation
- **Axial structure preserved**: The Gaussian z-profile is frozen in
- **Total flux conserved**: $\Phi = \int\int B_\phi \, dr\, dz = \text{constant}$

### Physical Significance

The frozen-in flux condition is fundamental to plasma confinement:

- **Magnetic confinement**: Plasma cannot easily cross field lines
- **Solar physics**: Field lines stretch with solar wind, creating the Parker spiral
- **FRC dynamics**: The closed field structure traps plasma inside
- **Flux conservation**: Sets limits on how fast configurations can change

When the frozen-in condition breaks down (magnetic reconnection), dramatic energy release can occur.

---
## 2. Configuration Setup

In [ ]:
# Standard imports
import jax
import jax.numpy as jnp
import jax.lax as lax
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML

# jax-frc imports
from jax_frc.configurations import FrozenFluxConfiguration
from jax_frc.solvers.explicit import EulerSolver

# Notebook utilities
import sys
sys.path.insert(0, '.')
from _shared import plot_comparison, plot_error, compute_metrics, metrics_summary, animate_overlay

# Plotting style
%matplotlib inline
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 11

In [ ]:
# === PHYSICS PARAMETERS ===

B_phi_0 = 1.0      # Peak toroidal field [T]
v_r = 0.1          # Radial expansion velocity [m/s]
eta = 1e-8         # Very small resistivity (Rm >> 1)
sigma_z = 0.3      # Axial Gaussian width [m]

# === GRID PARAMETERS (2D) ===

nr = 64            # High radial resolution
nz = 64            # High axial resolution for 2D structure
r_min = 0.2        # Inner radius [m]
r_max = 1.0        # Outer radius [m]
z_extent = 1.0     # Domain: z in [-z_extent, z_extent]

# === TIME PARAMETERS ===

# Advection timescale
L = r_max - r_min
tau_adv = L / v_r
print(f"Advection timescale: tau_adv = {tau_adv:.2f} s")

t_end = 0.5 * tau_adv    # Run for 0.5 advection times
dt = 1e-3 * tau_adv      # CFL-like timestep

print(f"Simulation time: t_end = {t_end:.2f} s")
print(f"Timestep: dt = {dt:.4f} s")
print(f"Expected steps: ~{int(t_end/dt)}")

In [ ]:
# Create configuration with 2D structure
config = FrozenFluxConfiguration(
    B_phi_0=B_phi_0,
    v_r=v_r,
    eta=eta,
    nr=nr,
    nz=nz,
    r_min=r_min,
    r_max=r_max,
    z_extent=z_extent,
)

# Build simulation components
geometry = config.build_geometry()
initial_state = config.build_initial_state(geometry)
model = config.build_model()
solver = EulerSolver()

# Add 2D Gaussian structure in z (modify initial state)
z_grid = geometry.z_grid
B_phi_2d = B_phi_0 * jnp.exp(-z_grid**2 / (2 * sigma_z**2))
B_modified = initial_state.B.at[:, :, 1].set(B_phi_2d)
initial_state = initial_state.replace(B=B_modified)

print(f"Grid: {geometry.nr} x {geometry.nz} (r x z)")
print(f"Domain: r in [{geometry.r_min:.2f}, {geometry.r_max:.2f}], z in [{geometry.z_min:.1f}, {geometry.z_max:.1f}]")
print(f"Resolution: dr = {geometry.dr:.4f}, dz = {geometry.dz:.4f}")
print(f"\nMagnetic Reynolds number: Rm = {config.magnetic_reynolds_number():.2e} (should be >> 1)")

### Visualize 2D Initial Condition

In [ ]:
# Get 2D grid coordinates
r = np.array(geometry.r_grid)
z = np.array(geometry.z_grid)

# Initial B_phi profile (2D)
Bphi_init = np.array(initial_state.B[:, :, 1])

# Initial v_r profile (should be uniform)
vr_init = np.array(initial_state.v[:, :, 0])

# Create figure with 2D contour and cross-sections
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# 2D contour plot of B_phi
im0 = axes[0, 0].contourf(z, r, Bphi_init, levels=20, cmap='viridis')
axes[0, 0].set_xlabel('z [m]', fontsize=12)
axes[0, 0].set_ylabel('r [m]', fontsize=12)
axes[0, 0].set_title('Initial $B_\\phi$ (2D)', fontsize=14)
plt.colorbar(im0, ax=axes[0, 0], label='$B_\\phi$ [T]')

# 2D contour plot of v_r
im1 = axes[0, 1].contourf(z, r, vr_init, levels=20, cmap='coolwarm')
axes[0, 1].set_xlabel('z [m]', fontsize=12)
axes[0, 1].set_ylabel('r [m]', fontsize=12)
axes[0, 1].set_title('Radial Velocity $v_r$ (2D)', fontsize=14)
plt.colorbar(im1, ax=axes[0, 1], label='$v_r$ [m/s]')

# Radial slice at z=0
z_mid_idx = geometry.nz // 2
r_1d = r[:, z_mid_idx]
axes[1, 0].plot(r_1d, Bphi_init[:, z_mid_idx], 'b-', linewidth=2)
axes[1, 0].axhline(B_phi_0, color='gray', linestyle='--', alpha=0.5, label=f'$B_0$ = {B_phi_0}')
axes[1, 0].set_xlabel('r [m]', fontsize=12)
axes[1, 0].set_ylabel('$B_\\phi$ [T]', fontsize=12)
axes[1, 0].set_title('Radial slice at z=0', fontsize=14)
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Axial slice at mid-r
r_mid_idx = geometry.nr // 2
z_1d = z[r_mid_idx, :]
axes[1, 1].plot(z_1d, Bphi_init[r_mid_idx, :], 'b-', linewidth=2)
axes[1, 1].axvline(-sigma_z, color='gray', linestyle=':', alpha=0.5)
axes[1, 1].axvline(sigma_z, color='gray', linestyle=':', alpha=0.5, label=f'$\\sigma_z$ = {sigma_z}')
axes[1, 1].set_xlabel('z [m]', fontsize=12)
axes[1, 1].set_ylabel('$B_\\phi$ [T]', fontsize=12)
axes[1, 1].set_title('Axial slice (Gaussian profile)', fontsize=14)
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Compute initial flux
flux_init = np.trapezoid(np.trapezoid(Bphi_init, z_1d, axis=1), r_1d)
print(f"Initial total toroidal flux: Phi_0 = {flux_init:.4f} T*m^2")

---
## 3. Run 2D Simulation

We evolve the plasma with uniform radial expansion. The 2D toroidal field should decrease according to flux conservation while preserving the axial Gaussian structure.

In [ ]:
# Time stepping setup
n_steps = int(t_end / dt)
save_interval = max(1, n_steps // 50)  # ~50 snapshots

print(f"Running {n_steps} steps with dt = {dt:.4f}")
print(f"Saving every {save_interval} steps")

In [ ]:
# Run simulation using lax.scan for efficiency
@jax.jit
def scan_step(state, _):
    new_state = solver.step(state, dt, model, geometry)
    return new_state, new_state.B[:, :, 1]  # Save full 2D B_phi

# Run simulation
print("Running 2D simulation...")
final_state, Bphi_history = lax.scan(scan_step, initial_state, None, length=n_steps)

# Convert to numpy and subsample for plotting
Bphi_history = np.array(Bphi_history)[::save_interval]
times = np.arange(0, n_steps, save_interval) * dt

# Add final state if not included
Bphi_final = np.array(final_state.B[:, :, 1])
if len(times) == 0 or times[-1] < t_end - dt:
    Bphi_history = np.concatenate([Bphi_history, Bphi_final[np.newaxis, :, :]], axis=0)
    times = np.append(times, final_state.time)

print(f"Simulation complete: t = {final_state.time:.4f} s")
print(f"Saved {len(times)} 2D snapshots")

---
## 4. Compare with 2D Analytic Solution

In [ ]:
# Compute 2D analytic solution at final time
# B_phi(r, z, t) = B_0 * (r_min / (r_min + v_r*t)) * exp(-z^2/(2*sigma_z^2))
r_inner_final = r_min + v_r * final_state.time
decay_factor = r_min / r_inner_final
Bphi_analytic = B_phi_0 * decay_factor * np.exp(-z**2 / (2 * sigma_z**2))

print(f"Inner radius: r_min = {r_min:.2f} m -> r(t) = {r_inner_final:.2f} m")
print(f"Expected B_phi decay factor: {decay_factor:.4f}")
print(f"Peak B_phi: initial = {B_phi_0:.3f} T, final = {B_phi_0 * decay_factor:.3f} T")

In [ ]:
# 2D comparison plots
fig, axes = plt.subplots(2, 3, figsize=(15, 9))

# Top row: Initial, Final Numerical, Final Analytic (2D contours)
vmax = B_phi_0
levels = np.linspace(0, vmax, 21)

im0 = axes[0, 0].contourf(z, r, Bphi_init, levels=levels, cmap='viridis')
axes[0, 0].set_title('Initial $B_\\phi$', fontsize=12)
axes[0, 0].set_ylabel('r [m]')
plt.colorbar(im0, ax=axes[0, 0])

im1 = axes[0, 1].contourf(z, r, Bphi_final, levels=levels, cmap='viridis')
axes[0, 1].set_title(f'Numerical $B_\\phi$ at t={final_state.time:.2f}s', fontsize=12)
plt.colorbar(im1, ax=axes[0, 1])

im2 = axes[0, 2].contourf(z, r, Bphi_analytic, levels=levels, cmap='viridis')
axes[0, 2].set_title(f'Analytic $B_\\phi$ at t={final_state.time:.2f}s', fontsize=12)
plt.colorbar(im2, ax=axes[0, 2])

# Bottom row: Error, Radial slice, Axial slice
error_2d = Bphi_final - Bphi_analytic
im3 = axes[1, 0].contourf(z, r, error_2d, levels=20, cmap='coolwarm')
axes[1, 0].set_title('Error (Num - Ana)', fontsize=12)
axes[1, 0].set_xlabel('z [m]')
axes[1, 0].set_ylabel('r [m]')
plt.colorbar(im3, ax=axes[1, 0], label='Error [T]')

# Radial slice comparison at z=0
axes[1, 1].plot(r_1d, Bphi_init[:, z_mid_idx], 'gray', linestyle=':', label='Initial')
axes[1, 1].plot(r_1d, Bphi_final[:, z_mid_idx], 'b-', linewidth=2, label='Numerical')
axes[1, 1].axhline(Bphi_analytic[0, z_mid_idx], color='orange', linestyle='--', 
                   linewidth=2, label=f'Analytic')
axes[1, 1].set_xlabel('r [m]')
axes[1, 1].set_ylabel('$B_\\phi$ [T]')
axes[1, 1].set_title('Radial slice at z=0')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

# Axial slice comparison at mid-r
axes[1, 2].plot(z_1d, Bphi_init[r_mid_idx, :], 'gray', linestyle=':', label='Initial')
axes[1, 2].plot(z_1d, Bphi_final[r_mid_idx, :], 'b-', linewidth=2, label='Numerical')
axes[1, 2].plot(z_1d, Bphi_analytic[r_mid_idx, :], 'r--', linewidth=2, label='Analytic')
axes[1, 2].set_xlabel('z [m]')
axes[1, 2].set_ylabel('$B_\\phi$ [T]')
axes[1, 2].set_title('Axial slice at mid-r')
axes[1, 2].legend()
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 2D Flux conservation check
flux_final = np.trapezoid(np.trapezoid(Bphi_final, z_1d, axis=1), r_1d)
flux_error = abs(flux_final - flux_init) / abs(flux_init)

print("2D Flux Conservation:")
print("=" * 50)
print(f"Initial flux: Phi_0 = {flux_init:.6f} T*m^2")
print(f"Final flux:   Phi  = {flux_final:.6f} T*m^2")
print(f"Relative error: {flux_error:.4%}")
print(f"Status: {'PASS' if flux_error < 0.01 else 'FAIL'} (threshold: 1%)")

In [ ]:
# Compute 2D validation metrics
metrics = compute_metrics(Bphi_final.ravel(), Bphi_analytic.ravel())
metrics['flux_conservation'] = flux_error

# Display with thresholds
thresholds = {
    'l2_error': 0.05,           # 5% relative L2 error
    'flux_conservation': 0.01,  # 1% flux conservation
}

print("\n2D Validation Metrics:")
print("=" * 50)
for name, value in metrics.items():
    threshold = thresholds.get(name, None)
    if threshold:
        status = 'PASS' if value <= threshold else 'FAIL'
        print(f"{name:20s}: {value:.4e}  (threshold: {threshold:.2e}) {status}")
    else:
        print(f"{name:20s}: {value:.4e}")

# Overall result
overall_pass = all(metrics[k] <= thresholds[k] for k in thresholds if k in metrics)
print("=" * 50)
print(f"Overall: {'PASS' if overall_pass else 'FAIL'}")

---
## 5. 2D Time Evolution Animation

In [ ]:
# 2D animation
from matplotlib.animation import FuncAnimation

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Initialize plots
levels = np.linspace(0, B_phi_0, 21)

def animate(frame):
    t = times[frame]
    
    # Clear and replot
    axes[0].clear()
    axes[0].contourf(z, r, Bphi_history[frame], levels=levels, cmap='viridis')
    axes[0].set_xlabel('z [m]')
    axes[0].set_ylabel('r [m]')
    axes[0].set_title(f'Numerical $B_\\phi$ (t={t:.3f}s)')
    
    # Analytic at time t
    decay = r_min / (r_min + v_r * t)
    Bphi_ana_t = B_phi_0 * decay * np.exp(-z**2 / (2 * sigma_z**2))
    
    axes[1].clear()
    axes[1].contourf(z, r, Bphi_ana_t, levels=levels, cmap='viridis')
    axes[1].set_xlabel('z [m]')
    axes[1].set_ylabel('r [m]')
    axes[1].set_title(f'Analytic $B_\\phi$ (t={t:.3f}s)')
    
    return []

anim = FuncAnimation(fig, animate, frames=len(times), interval=200, blit=False)
plt.close()

HTML(anim.to_jshtml())

---
## 6. Interactive 2D Exploration

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

def explore_time_2d(time_idx):
    """Show 2D solution at a specific snapshot."""
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    t = times[time_idx]
    Bphi_num = Bphi_history[time_idx]
    
    # Analytic at time t
    decay = r_min / (r_min + v_r * t)
    Bphi_ana = B_phi_0 * decay * np.exp(-z**2 / (2 * sigma_z**2))
    
    # 2D numerical
    levels = np.linspace(0, B_phi_0, 21)
    im0 = axes[0, 0].contourf(z, r, Bphi_num, levels=levels, cmap='viridis')
    axes[0, 0].set_xlabel('z [m]')
    axes[0, 0].set_ylabel('r [m]')
    axes[0, 0].set_title(f'Numerical $B_\\phi$ at t = {t:.4f} s')
    plt.colorbar(im0, ax=axes[0, 0])
    
    # 2D analytic
    im1 = axes[0, 1].contourf(z, r, Bphi_ana, levels=levels, cmap='viridis')
    axes[0, 1].set_xlabel('z [m]')
    axes[0, 1].set_ylabel('r [m]')
    axes[0, 1].set_title(f'Analytic $B_\\phi$ at t = {t:.4f} s')
    plt.colorbar(im1, ax=axes[0, 1])
    
    # Flux over time
    flux_history = [np.trapezoid(np.trapezoid(Bphi_history[i], z_1d, axis=1), r_1d) 
                    for i in range(len(times))]
    axes[1, 0].plot(times, flux_history, 'g-', linewidth=2, label='Numerical')
    axes[1, 0].axhline(flux_init, color='gray', linestyle='--', alpha=0.7, label='Initial')
    axes[1, 0].axvline(t, color='red', linestyle=':', alpha=0.7)
    axes[1, 0].set_xlabel('Time [s]')
    axes[1, 0].set_ylabel('Flux $\\Phi$ [T*m^2]')
    axes[1, 0].set_title('2D Flux Conservation')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # Axial slice at mid-r
    axes[1, 1].plot(z_1d, Bphi_init[r_mid_idx, :], 'gray', linestyle=':', label='Initial')
    axes[1, 1].plot(z_1d, Bphi_num[r_mid_idx, :], 'b-', linewidth=2, label='Numerical')
    axes[1, 1].plot(z_1d, Bphi_ana[r_mid_idx, :], 'r--', linewidth=2, label='Analytic')
    axes[1, 1].set_xlabel('z [m]')
    axes[1, 1].set_ylabel('$B_\\phi$ [T]')
    axes[1, 1].set_title('Axial profile (Gaussian preserved)')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print flux
    current_flux = np.trapezoid(np.trapezoid(Bphi_num, z_1d, axis=1), r_1d)
    flux_err = abs(current_flux - flux_init) / flux_init
    print(f"Current flux: {current_flux:.6f} T*m^2  |  Flux error: {flux_err:.4%}")
    print(f"B_phi decay factor: {decay:.4f}")

# Time slider
time_slider = widgets.IntSlider(
    value=0, min=0, max=len(times)-1, step=1,
    description='Snapshot:',
    continuous_update=False
)

ui = widgets.VBox([
    widgets.HTML('<h4>2D Time Evolution Explorer</h4>'),
    widgets.HTML('<p>Drag the slider to see the 2D solution. Watch B_phi decrease while the Gaussian z-profile is preserved!</p>'),
    time_slider,
])

interactive_output = widgets.interactive_output(explore_time_2d, {'time_idx': time_slider})

display(ui, interactive_output)

---
## Summary

This notebook demonstrated **2D frozen-in flux advection** in ideal MHD:

1. **Physics**: In the ideal MHD limit ($R_m \gg 1$), magnetic field is "frozen" into the plasma. Field lines move with the fluid and cannot slip through it.

2. **Alfven's theorem**: Magnetic flux through any surface moving with the plasma is conserved:
   $$\frac{D\Phi}{Dt} = 0$$

3. **2D Analytic solution**: For uniform radial expansion with Gaussian axial structure:
   - Radial decay: $B_\phi \propto r_0/(r_0 + v_r t)$ from flux conservation
   - Axial profile: Gaussian structure is preserved (frozen in)

4. **Validation**: The numerical solution conserves 2D flux within the specified tolerance, confirming correct implementation of ideal MHD advection.

### Physical Significance

The frozen-in flux condition is fundamental to:
- **Magnetic confinement**: Plasma trapped inside closed field lines
- **FRC formation**: Flux trapping during field reversal  
- **Solar wind**: Parker spiral from rotating sun + radial outflow
- **Magnetic reconnection**: When frozen-in breaks down, energy is released

### Try Next

- Increase `eta` to see flux leakage (transition toward $R_m \sim 1$)
- Vary `sigma_z` to see different axial structures preserved
- Compare with the **magnetic diffusion** notebook to see the opposite limit ($R_m \ll 1$)